In [1]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------
# 1️⃣ Load all required CSVs for both years
# ------------------------------------------------------------
enrol_2324 = pd.read_csv('/Users/sahitipotini/Downloads/UDISE_2023-24/100_enr1.csv')
enrol_2425 = pd.read_csv('/Users/sahitipotini/Downloads/UDISE_2024-25/100_enr1.csv')
teachers_2324 = pd.read_csv('/Users/sahitipotini/Downloads/UDISE_2023-24/100_tch.csv')
teachers_2425 = pd.read_csv('/Users/sahitipotini/Downloads/UDISE_2024-25/100_tch.csv')
fac_2324 = pd.read_csv('/Users/sahitipotini/Downloads/UDISE_2023-24/100_fac.csv')
fac_2425 = pd.read_csv('/Users/sahitipotini/Downloads/UDISE_2024-25/100_fac.csv')
profile_2324 = pd.read_csv('/Users/sahitipotini/Downloads/UDISE_2023-24/100_prof1.csv')
profile_2425 = pd.read_csv('/Users/sahitipotini/Downloads/UDISE_2024-25/100_prof1.csv')

# ------------------------------------------------------------
# 2️⃣ Clean and select relevant columns
# ------------------------------------------------------------
teachers_2324 = teachers_2324[['pseudocode', 'total_tch', 'trained_cwsn']].rename(columns={
    'total_tch': 'total_tch_2324', 'trained_cwsn': 'trained_cwsn_2324'
})
teachers_2425 = teachers_2425[['pseudocode', 'total_tch', 'trained_cwsn']].rename(columns={
    'total_tch': 'total_tch_2425', 'trained_cwsn': 'trained_cwsn_2425'
})

fac_2324 = fac_2324[['pseudocode',
                     'total_boys_func_toilet', 'total_girls_func_toilet',
                     'total_boys_toilet', 'total_girls_toilet',
                     'electricity_availability', 'library_availability']].rename(
    columns=lambda x: x + '_2324' if x != 'pseudocode' else x
)
fac_2425 = fac_2425[['pseudocode',
                     'total_boys_func_toilet', 'total_girls_func_toilet',
                     'total_boys_toilet', 'total_girls_toilet',
                     'electricity_availability', 'library_availability']].rename(
    columns=lambda x: x + '_2425' if x != 'pseudocode' else x
)

profile_2324 = profile_2324[['pseudocode', 'district', 'school_type', 'management', 'rural_urban']]

# ------------------------------------------------------------
# 3️⃣ Compute total enrolment for each year
# ------------------------------------------------------------
enrol_cols = ['pseudocode'] + [c for c in enrol_2324.columns if c.startswith(('c', 'cpp_'))]
enrol_2324 = enrol_2324[enrol_cols]
enrol_2425 = enrol_2425[enrol_cols]

enrol_2324['total_enrol_2324'] = enrol_2324.drop('pseudocode', axis=1).sum(axis=1)
enrol_2425['total_enrol_2425'] = enrol_2425.drop('pseudocode', axis=1).sum(axis=1)

enrol_2324 = enrol_2324[['pseudocode', 'total_enrol_2324']]
enrol_2425 = enrol_2425[['pseudocode', 'total_enrol_2425']]

# ------------------------------------------------------------
# 4️⃣ Merge teacher and facility data across years
# ------------------------------------------------------------
merged_teachers = pd.merge(teachers_2324, teachers_2425, on='pseudocode', how='outer')
merged_fac = pd.merge(fac_2324, fac_2425, on='pseudocode', how='outer')

# Average & change in teachers
merged_teachers['avg_total_tch'] = merged_teachers[['total_tch_2324', 'total_tch_2425']].mean(axis=1)
merged_teachers['change_total_tch'] = merged_teachers['total_tch_2425'] - merged_teachers['total_tch_2324']

# Average & change in functional toilets
for col in ['total_boys_func_toilet', 'total_girls_func_toilet', 'total_boys_toilet', 'total_girls_toilet']:
    merged_fac[f'avg_{col}'] = merged_fac[[col + '_2324', col + '_2425']].mean(axis=1)
    merged_fac[f'change_{col}'] = merged_fac[col + '_2425'] - merged_fac[col + '_2324']

# Compute average functional toilet ratio
merged_fac['avg_functional_toilet_ratio'] = (
    (merged_fac['avg_total_boys_func_toilet'] + merged_fac['avg_total_girls_func_toilet']) /
    (merged_fac['avg_total_boys_toilet'] + merged_fac['avg_total_girls_toilet'])
)

# Compute change in functional toilet ratio
merged_fac['change_functional_toilet_ratio'] = (
    ((merged_fac['total_boys_func_toilet_2425'] + merged_fac['total_girls_func_toilet_2425']) /
     (merged_fac['total_boys_toilet_2425'] + merged_fac['total_girls_toilet_2425'])) -
    ((merged_fac['total_boys_func_toilet_2324'] + merged_fac['total_girls_func_toilet_2324']) /
     (merged_fac['total_boys_toilet_2324'] + merged_fac['total_girls_toilet_2324']))
)

# ------------------------------------------------------------
# 5️⃣ Merge everything into one master DataFrame
# ------------------------------------------------------------
df = (enrol_2324.merge(enrol_2425, on='pseudocode', how='inner')
                .merge(merged_teachers, on='pseudocode', how='left')
                .merge(merged_fac, on='pseudocode', how='left')
                .merge(profile_2324, on='pseudocode', how='left'))

# ------------------------------------------------------------
# 6️⃣ Derived Metrics
# ------------------------------------------------------------
df['retention_rate'] = (df['total_enrol_2425'] / df['total_enrol_2324']) * 100
df['pupil_teacher_ratio'] = df['total_enrol_2324'] / df['avg_total_tch']

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.loc[df['pupil_teacher_ratio'] > 200, 'pupil_teacher_ratio'] = np.nan

# ------------------------------------------------------------
# 7️⃣ Keep final analysis columns
# ------------------------------------------------------------
school_level = df[['pseudocode', 'district',
                   'retention_rate', 'pupil_teacher_ratio',
                   'avg_functional_toilet_ratio', 'change_functional_toilet_ratio',
                   'avg_total_tch', 'change_total_tch',
                   'school_type', 'management', 'rural_urban']]

# ------------------------------------------------------------
# 8️⃣ Aggregate to district level
# ------------------------------------------------------------
district_level = school_level.groupby('district', as_index=False).agg({
    'retention_rate': 'mean',
    'pupil_teacher_ratio': 'mean',
    'avg_functional_toilet_ratio': 'mean',
    'change_functional_toilet_ratio': 'mean',
    'avg_total_tch': 'mean',
    'change_total_tch': 'mean'
}).round(2)

# ------------------------------------------------------------
# 9️⃣ Save both outputs
# ------------------------------------------------------------
school_level.to_csv('school_retention_analysis_school_level.csv', index=False)
district_level.to_csv('school_retention_analysis_district_level.csv', index=False)

print("✅ Final datasets generated successfully!")
print("➡️ school_retention_analysis_school_level.csv")
print("➡️ school_retention_analysis_district_level.csv")


✅ Final datasets generated successfully!
➡️ school_retention_analysis_school_level.csv
➡️ school_retention_analysis_district_level.csv
